In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from nbainjuries import injury

import warnings
import duckdb
import os
import requests

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}

print("Current working dir:", cwd)
print("RUN_LOCATION =", RUN_LOCATION)

Current working dir: C:/Users/Rodolfo/Jupyter_files/FantasyBasketball/notebooks
RUN_LOCATION = local


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

folders = os.listdir('../tables/')
df = pd.DataFrame()
for yr in folders:
    df_temp = pd.read_csv(f"../tables/{yr}/season_gamelogs.csv")
    df_temp.insert(0, 'Season', int(yr))
    df = pd.concat([df, df_temp])
df['Date'] = pd.to_datetime(df.Date)
df = df.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
df['STL_BLK'] = df.STL + df.BLK
df['PR'] = df.PTS + df.REB 
df['PA'] = df.PTS + df.AST
df['RA'] = df.REB + df.AST
df['PRA'] = df.PTS + df.REB + df.AST

# # CONTROL DATE TO GO BACK AND RELOAD HISTORICAL DATA
# df = df[(df.Date != now)]

NameError: name 'timedelta' is not defined

In [ ]:
%run ./common_utils.ipynb

In [4]:
df_gms = pd.read_csv(f"../tables/{year}/nba_schedule.csv")
df_gms = df_gms[(df_gms.Date == now)]
tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
df_gms['gm_id'] = df_gms.AwayABV + "_" + df_gms.HomeABV
df_gms['gm_id2'] = df_gms.HomeABV + "_" + df_gms.AwayABV
gms_today = df_gms.gm_id.tolist() + df_gms.gm_id2.tolist()
df_gms = df_gms.drop(['gm_id', 'gm_id2'], axis=1)
display(df_gms)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
347,2025-12-07,12:00p,Orlando Magic,ORL,New York Knicks,NYK,Madison Square Garden (IV),0,0,0
348,2025-12-07,3:30p,Boston Celtics,BOS,Toronto Raptors,TOR,Scotiabank Arena,0,0,0
349,2025-12-07,6:00p,Denver Nuggets,DEN,Charlotte Hornets,CHO,Spectrum Center,0,0,0
350,2025-12-07,6:00p,Portland Trail Blazers,POR,Memphis Grizzlies,MEM,FedExForum,0,0,0
351,2025-12-07,7:00p,Golden State Warriors,GSW,Chicago Bulls,CHI,United Center,1,0,0
352,2025-12-07,7:30p,Los Angeles Lakers,LAL,Philadelphia 76ers,PHI,Xfinity Mobile Arena,0,0,0
353,2025-12-07,8:00p,Oklahoma City Thunder,OKC,Utah Jazz,UTA,Delta Center,0,0,0


In [5]:
df_teams = pd.read_csv("../src/team_info_xref.csv")

df_inj = injury.get_reportdata(datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]), return_df=True)
df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
df_inj['Player'] = df_inj['Player'].str.strip()
df_inj['Date'] = pd.to_datetime(df_inj['Date'])
df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                         JOIN df_teams ON df_inj.Team = df_teams.Team
                         WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()

df_best_out = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Fpts DESC) as Off_Rk FROM
                              (SELECT Team, Player, Pos, AVG(Fpts) as Fpts FROM df 
                              WHERE Season = {year} AND Fpts > 0 AND Pos != 'None'
                              GROUP BY Team, Player, Pos)""").fetchdf()
df_best_out = con.execute(f"""SELECT * EXCLUDE(t2.Team, t2.Player) FROM df_best_out t1 
                              JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                              ORDER BY Fpts DESC""").fetchdf()
display(df_best_out)

Validated Injury-Report_2025-12-07_11PM.


,Team,Player,Pos,Fpts,Off_Rk,Date,Status,Reason
0,OKC,Shai Gilgeous-Alexander,PG,31.613636,4,2025-12-07,Out,Injury/Illness - Left Elbow; Bursitis
1,NYK,Karl-Anthony Towns,C,28.636364,4,2025-12-07,Out,Injury/Illness - Left Calf; Tightness
2,UTA,Lauri Markkanen,PF,26.000000,5,2025-12-07,Out,Injury/Illness - N/A; Illness
3,GSW,Stephen Curry,PG,25.343750,8,2025-12-07,Out,Injury/Illness - Left Quadriceps; Contusion
4,UTA,Walker Kessler,C,25.200000,7,2025-12-07,Out,Injury/Illness - Left Shoulder; Injury Recovery
5,CHO,LaMelo Ball,PG,25.156250,9,2025-12-07,Out,Injury/Illness - Left Ankle; Bone Bruise
6,POR,Jrue Holiday,PG,24.125000,12,2025-12-07,Out,Injury/Illness - Right Calf; Strain
7,OKC,Isaiah Hartenstein,C,22.777778,13,2025-12-07,Out,Injury/Illness - Right Soleus; Strain
8,POR,Donovan Clingan,C,20.761905,18,2025-12-07,Out,Injury/Illness - Left Lower Leg; Contusion
9,TOR,RJ Barrett,SF,19.764706,15,2025-12-07,Out,Injury/Illness - Right Knee; Sprain


In [6]:
def get_game_odds():

    dk_tm_mapping = {
                    "ATL Hawks": "ATL", "BKN Nets": "BRK", "BOS Celtics": "BOS", "CHA Hornets": "CHO", "CHI Bulls": "CHI",
                    "CLE Cavaliers": "CLE", "DAL Mavericks": "DAL", "DEN Nuggets": "DEN", "DET Pistons": "DET", 
                    "GS Warriors": "GSW", "HOU Rockets": "HOU", "IND Pacers": "IND", "LA Clippers": "LAC", 
                    "LA Lakers": "LAL", "MEM Grizzlies": "MEM", "MIA Heat": "MIA", "MIL Bucks": "MIL", "MIN Timberwolves": "MIN",
                    "NO Pelicans": "NOP", "NY Knicks": "NYK", "OKC Thunder": "OKC", "ORL Magic": "ORL", "PHI 76ers": "PHI",
                    "PHO Suns": "PHO", "POR Trail Blazers": "POR", "SA Spurs": "SAS", "SAC Kings": "SAC", "TOR Raptors": "TOR",
                    "UTA Jazz": "UTA", "WAS Wizards": "WAS", "GSW Warriors": "GSW", "LAL Lakers": "LAL", "NOP Pelicans": "NOP",
                    "NYK Knicks": "NYK", "PHX Suns": "PHO", "SAS Spurs": "SAS"
                    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }
    response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C4511&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%274511%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%274511%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
    if response.status_code != 200:
        raise Exception('Bad Request')

    ids = []
    rows = []
    for i in range(len(response.json()['selections'])):
        if 'HC' in response.json()['selections'][i]['id'] or 'OU' in response.json()['selections'][i]['id']:
            ids.append(response.json()['selections'][i])
    for i in range(0, len(ids) - 1, 4):
        team1 = ids[i]['label']
        spread1 = ids[i]['points']
        team2 = ids[i+1]['label']
        spread2 = ids[i+1]['points']
        total = ids[i+2]['points']
        rows.append({"Team": team1, "Opp": team2, "Spread": spread1, "Total": total})
        rows.append({"Team": team2, "Opp": team1, "Spread": spread2, "Total": total})
    df_tm_bets = pd.DataFrame(rows)
    
    for key, value in dk_tm_mapping.items():
        df_tm_bets['Team'] = np.where(df_tm_bets.Team == key, value, df_tm_bets.Team)
        df_tm_bets['Opp'] = np.where(df_tm_bets.Opp == key, value, df_tm_bets.Opp)

    df_tm_bets['gm_id'] = df_tm_bets.Team + "_" + df_tm_bets.Opp
    df_tm_bets = df_tm_bets[(df_tm_bets.gm_id.isin(gms_today))].drop(['Opp', 'gm_id'], axis=1)
    
    return df_tm_bets

def get_sportsbook():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://sportsbook.draftkings.com/",
        "Origin": "https://sportsbook.draftkings.com"
    }

    dk_cats = {"PTS": 12488, "AST": 12495, "REB": 12492, "STL": 13508, "BLK": 13780, "STL_BLK": 13781, "TPM": 12497, 
               "PA": 9973, "PR": 9976, "RA": 9974, "PRA": 5001}
    df_lines = pd.DataFrame()
    for key, value in dk_cats.items():
        response = requests.get(f"https://sportsbook-nash.draftkings.com/sites/US-SB/api/sportscontent/controldata/league/leagueSubcategory/v1/markets?isBatchable=false&templateVars=42648%2C{value}&eventsQuery=%24filter%3DleagueId%20eq%20%2742648%27%20AND%20clientMetadata%2FSubcategories%2Fany%28s%3A%20s%2FId%20eq%20%27{value}%27%29&marketsQuery=%24filter%3DclientMetadata%2FsubCategoryId%20eq%20%27{value}%27%20AND%20tags%2Fall%28t%3A%20t%20ne%20%27SportcastBetBuilder%27%29&include=Events&entity=events", headers=headers)
        if response.status_code != 200:
            raise Exception('Bad Request')

        plyr_names = []
        pnt_lines = []
        for i in response.json()['selections']:
            plyr_names.append(i['participants'][0]['name'])
            pnt_lines.append(i['points'])
        df_dk = pd.DataFrame({"Player": plyr_names, f"{key}_line": pnt_lines}).drop_duplicates().reset_index(drop=True)

        if df_lines.empty:
            df_lines = df_dk
        else:
            df_lines = pd.merge(df_lines, df_dk, on="Player", how="outer")

    df_lines = pd.merge(pd.read_csv(f"../tables/{year}/plyr_pos_xref.csv"), df_lines, on='Player', how='right')
    df_lines = pd.merge(df_lines, get_game_odds(), on='Team')
    df_lines.insert(0, 'Date', pd.to_datetime(now))

    partition_save_df(df_lines, f"../tables/{year}/parlay_lines.csv") 
    display(df_lines)

    return df_lines

df_lines = get_sportsbook()

../tables/2025/parlay_lines.csv saved!


,Date,Team,Player,Pos,PTS_line,AST_line,REB_line,STL_line,BLK_line,STL_BLK_line,TPM_line,PA_line,PR_line,RA_line,PRA_line,Spread,Total


In [ ]:
def pick_finder(stat, collect=False):
    df_actuals = pd.read_csv(f"../tables/{year}/parlay_actuals.csv")
    
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = {year} AND {stat} > 0 AND Pos != 'None'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = {year} AND {stat} > 0 AND Pos != 'None'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 27 AND Season = {year} AND Pos = '{pos}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})""").fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 27 AND Season = {year} AND Pos = '{pos}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()

        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date >= '2023-10-23'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
        
        # Add hit odds (df_actuals)
        df_hit_odds = con.execute(f"""WITH overs AS 
                                        (SELECT Team, Player, SUM({stat}_Diff) as O_Diff, count(*) as Hits FROM df_actuals 
                                        WHERE {stat}_Result = 'O' 
                                        GROUP BY Team, Player)

                                        SELECT overs.Team, overs.Player, Hits, Misses,
                                        O_Diff + U_Diff as {stat}_P_Diff,
                                        hits / (misses + hits) AS {stat}_Hit_Pct, misses / (misses + hits) AS Miss_Pct FROM

                                       (SELECT Team, Player, SUM({stat}_Diff) as U_Diff, count(*) as Misses FROM df_actuals 
                                       WHERE {stat}_Result = 'U' AND {stat}_line IS NOT NULL
                                       GROUP BY Team, Player) unders

                                       JOIN overs ON overs.Team = unders.Team AND overs.Player = unders.Player
                                       ORDER BY {stat}_Hit_Pct DESC""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.*, df_hit_odds.{stat}_Hit_Pct, df_hit_odds.{stat}_P_Diff FROM df_picks LEFT JOIN df_hit_odds 
                                   ON df_picks.Team = df_hit_odds.Team AND df_picks.Player = df_hit_odds.Player""").fetchdf()     
        
        # Add Spread and Total
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE({stat}_Hit_Pct, {stat}_P_Diff), df_lines.{stat}_line, {stat}_Hit_Pct, {stat}_P_Diff, 
                                   Spread, Total FROM df_picks LEFT JOIN df_lines 
                                   ON df_picks.Team = df_lines.Team AND df_picks.Player = df_lines.Player
                                   WHERE {stat}_line IS NOT NULL
                                   ORDER BY Def_Rk DESC""").fetchdf()
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [ ]:
for i in categories:
    print(f"==={i}===")
    pick_finder(i)

In [9]:
df_stats = pd.DataFrame()
for i in categories:
    print(f"==={i}===")
    df_temp = pick_finder(i, collect=True)
    df_stats = pd.concat([df_stats, df_temp], ignore_index=True)
    
df_stats = con.execute("SELECT Date, Team, B2B, Spread, Total, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk, Spread, Total) FROM df_stats").fetchdf()
df_save = pd.DataFrame()
for player in df_stats.Player.unique():
    df_temp = df_stats[(df_stats.Player == player)]
    for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
        df_temp[col] = df_temp[col].ffill()
        df_temp[col] = df_temp[col].backfill()
    df_temp = df_temp.drop_duplicates()
    df_save = pd.concat([df_save, df_temp])
print('Saving for Date:', now)
partition_save_df(df_save, f"../tables/{year}/parlay_stats.csv")
display(df_save)

===PTS===
PG
SG
SF
PF
C
===AST===
PG
SG
SF
PF
C
===REB===
PG
SG
SF
PF
C
===PR===
PG
SG
SF
PF
C
===PA===
PG
SG
SF
PF
C
===RA===
PG
SG
SF
PF
C
===PRA===
PG
SG
SF
PF
C
===TPM===
PG
SG
SF
PF
C
===STL===
PG
SG
SF
PF
C
===BLK===
PG
SG
SF
PF
C
===STL_BLK===
PG
SG
SF
PF
C
Saving for Date: 2025-12-02
../tables/2025/parlay_stats.csv saved!


""


# REPOPULATE DATA

In [3]:
def pick_finder2(stat, season, now, collect=False):    
    df_gms = pd.read_csv(f"../tables/{season}/nba_schedule.csv")
    df_gms['Date'] = pd.to_datetime(df_gms.Date)
    df_gms = df_gms[(df_gms.Date == now)]
    tms_today = df_gms.AwayABV.tolist() + df_gms.HomeABV.tolist()
    

    if now not in ['2023-05-02']:        
        df_inj = injury.get_reportdata(datetime.strptime(now, "%Y-%m-%d"), return_df=True)
        df_inj = df_inj.rename(columns={"Game Date": "Date", "Player Name": "Player", "Current Status": "Status"})
        df_inj['Player'] = df_inj.Player.str.split(",").str[1] + " " + df_inj.Player.str.split(",").str[0]
        df_inj['Player'] = df_inj['Player'].str.strip()
        df_inj['Date'] = pd.to_datetime(df_inj['Date'])
        df_inj = con.execute(f"""SELECT Date, ABV as Team, Player, Status, Reason FROM df_inj 
                             JOIN df_teams ON df_inj.Team = df_teams.Team
                             WHERE Date = '{now}' AND Status = 'Out'""").fetchdf()
    else:
        df_inj = pd.DataFrame(columns=['Date', 'Team', 'Player', 'Status', 'Reason'])
    
    df_mtch = df_gms[['AwayABV', 'HomeABV', 'AwayB2B', 'HomeB2B']]
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Team', 'Opp', 'B2B']]
    df_mtch = pd.concat([df_mtch, df_mtch2])
    
    df_rk = con.execute(f"""SELECT *, RANK() OVER (PARTITION BY Pos ORDER BY Off_{stat} DESC) as Off_Rk FROM
                            (SELECT Team, Player, Pos, AVG({stat}) as Off_{stat} FROM df 
                            WHERE Season = {season} AND Pos != 'None' AND Date <= '{now}'
                            AND Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                            GROUP BY Team, Player, Pos)""").fetchdf()
    df_rk_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Team, Player, Pos, {stat} AS stat_val,
                                    ROW_NUMBER() OVER (PARTITION BY Team, Player ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE Season = {season} AND Pos != 'None' AND Date <= '{now}'
                                    AND Team IN ({str(tms_today).replace('[','').replace(']','')})
                            )
                            SELECT Team, Player, Pos, AVG(stat_val) AS Off_L5_{stat},
                            FROM last5
                            WHERE rn <= 5
                            GROUP BY Team, Player, Pos
                            """).fetchdf()

    df_rk = con.execute(f"""SELECT df_rk.Team, df_rk.Player, df_rk.Pos, Off_{stat}, Off_L5_{stat}, Off_Rk FROM df_rk JOIN df_rk_l5 
                            ON df_rk.Team = df_rk_l5.Team AND df_rk.Player = df_rk_l5.Player""").fetchdf()
    df_save = pd.DataFrame()
    for pos in ['PG', 'SG', 'SF', 'PF', 'C']:
        print(pos)
        df_def = con.execute(f"""SELECT Team, '{pos}' AS Pos, Def_{stat}, Def_Rk FROM
                                 (SELECT *, RANK() OVER (ORDER BY Def_{stat}) as Def_Rk FROM
                                 (SELECT Opp as Team, AVG({stat}) as Def_{stat} FROM df 
                                 WHERE MP >= 18 AND Season = {season} AND Pos = '{pos}' AND Date <= '{now}'
                                 GROUP BY Opp
                                 ORDER BY AVG(Fpts) DESC))
                                 WHERE Team IN ({str(tms_today).replace("[", "").replace("]", "")})
                                 """).fetchdf()

        df_def_l5 = con.execute(f"""WITH last5 AS (
                                    SELECT Opp AS Team, {stat} AS stat_val, 
                                    ROW_NUMBER() OVER (PARTITION BY Opp ORDER BY Date DESC) AS rn
                                    FROM df
                                    WHERE MP >= 18 AND Season = {season} AND Pos = '{pos}' AND Date <= '{now}'
                                    )

                                    SELECT Team, '{pos}' AS Pos, AVG(stat_val) AS Def_L5_{stat} FROM last5
                                    WHERE rn <= 5
                                    GROUP BY Team
                                    HAVING Team IN ({str(tms_today).replace('[','').replace(']','')})
                                    """).fetchdf()
        df_def = con.execute(f"""SELECT df_def.Team, df_def.Pos, Def_{stat}, Def_L5_{stat}, Def_Rk FROM df_def 
                               JOIN df_def_l5 ON df_def.Team = df_def_l5.Team""").fetchdf()
        # Piece together the current matchups with offensive rankings vs defensive rankings
        df_picks = con.execute(f"""SELECT df_mtch.*, df_rk.* EXCLUDE(Team) FROM df_mtch 
                                  JOIN df_rk ON df_mtch.Team = df_rk.Team
                                  WHERE Pos = '{pos}'""").fetchdf()
        df_picks = con.execute(f"""SELECT df_picks.* EXCLUDE(Opp, Pos), Opp, df_def.Def_{stat}, df_def.Def_L5_{stat}, 
                                   df_def.Def_Rk FROM df_picks 
                                   JOIN df_def ON df_picks.Opp = df_def.Team""").fetchdf()
        df_picks['Rk_Diff'] = df_picks['Def_Rk'] - df_picks['Off_Rk']
        df_picks = con.execute(f"""SELECT t1.* EXCLUDE(Rk_Diff) FROM df_picks t1
                                LEFT JOIN df_inj t2 ON t1.Team = t2.Team AND t1.Player = t2.Player
                                WHERE Status IS NULL
                                ORDER BY Rk_Diff DESC""").fetchdf()
        # head to head
        df_h2h = pd.DataFrame()
        for index, row in df_picks.iterrows():
            query = f"""SELECT * EXCLUDE(Gms) FROM
                       (SELECT Player, COUNT(*) as Gms, AVG({stat}) AS AVG_{stat}_H2H FROM df 
                       WHERE Player = ? AND Opp = ? AND Date <= '{now}'
                       GROUP BY Player)
                       WHERE Gms >= 4"""
            df_temp = con.execute(query, [row["Player"], row["Opp"]]).fetchdf()
            df_h2h = pd.concat([df_h2h, df_temp])
        try:
            df_picks = con.execute(f"""SELECT Team, B2B, t1.Player, Off_{stat}, Off_L5_{stat}, t2.AVG_{stat}_H2H, Off_Rk, 
                                   t1.* EXCLUDE(Team, B2B, Player, Off_{stat}, Off_L5_{stat}, Off_Rk)
                                   FROM df_picks t1 LEFT JOIN df_h2h t2 
                                   ON t1.Player = t2.Player""").fetchdf()
        except:
            pass
    
        df_save = pd.concat([df_save, df_picks], ignore_index=True)
        if collect == False:
            display(df_picks)
    if collect == True:
        df_save.insert(0, 'Date', pd.to_datetime(now))
        return df_save

In [4]:
def repopulate_old_szns(season):
    schd = pd.read_csv(f"../tables/{season}/nba_schedule.csv")
    schd['Date'] = pd.to_datetime(schd.Date)
    schd = schd[(schd.Date >= '2023-05-02')]
    df_lines = pd.read_csv(f"../tables/{season}/parlay_lines.csv")
    for date in schd.Date.unique().tolist():
        dt_str = date.strftime("%Y-%m-%d")
        print(dt_str)
        now = dt_str

        df_stats = pd.DataFrame()
        for stat in ['PTS']:
            print(f"==={stat}===")
            df_temp = pick_finder2(stat, season, now, collect=True)
            df_stats = pd.concat([df_stats, df_temp], ignore_index=True)

        df_stats = con.execute("SELECT Date, Team, B2B, Player, Opp, * EXCLUDE(Date, Team, B2B, Player, Opp, Off_Rk, Def_Rk) FROM df_stats").fetchdf()
        df_save = pd.DataFrame()
        for player in df_stats.Player.unique():
            df_temp = df_stats[(df_stats.Player == player)]
            for col in df_temp.columns.difference(['Date', 'Team', 'Player']):
                df_temp[col] = df_temp[col].ffill()
                df_temp[col] = df_temp[col].backfill()
            df_temp = df_temp.drop_duplicates()
            df_save = pd.concat([df_save, df_temp])
        print('Saving for Date:', now)
#         display(df_save)
        partition_save_df(df_save, f"../tables/{season}/parlay_stats.csv")